In [ ]:
!pip install autoprognosis
!pip install tabulate

In [ ]:
import sys
import warnings
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import synthcity.logger as log
from sklearn.mixture import GaussianMixture as GMM

from datasets import get_dataset

warnings.filterwarnings("ignore", category=FutureWarning)

warnings.filterwarnings("ignore")
log.add(sink=sys.stderr, level="DEBUG")

out_dir = Path("workspace")
out_dir.mkdir(parents=True, exist_ok=True)

In [ ]:
from synthcity.plugins.core.models.time_to_event import (
    get_model_template as get_tte_model_template,
)


class TabularGMM:
    def __init__(self, components: int = 100, random_state: int = 0):
        self.model = GMM(100, covariance_type="full", random_state=random_state)
        self.tte_regressor = get_tte_model_template("survival_function_regression")()

    def fit(self, X, T, E):
        self.model.fit(X)
        self.tte_regressor.fit(X, T, E)

        self.E = E
        self.count = len(X)
        self.columns = X.columns

    def generate(self, count: int = None):
        if count is None:
            count = self.count
        sampled, _ = self.model.sample(count)
        sampled = pd.DataFrame(sampled, columns=self.columns)

        E = self.E.reset_index(drop=True).head(count)

        T = pd.Series(self.tte_regressor.predict_any(sampled, E,))
        return sampled, T, E

In [ ]:
from autoprognosis.plugins.prediction.risk_estimation import RiskEstimation
from autoprognosis.utils.metrics import generate_score, print_score
from autoprognosis.utils.tester import evaluate_survival_estimator
from sklearn.model_selection import train_test_split
from synthcity.utils.serialization import dataframe_hash, load_from_file, save_to_file

log.remove()
headers = ["dataset", "cindex", "brier score"]

scores = []
for ref_df in ["aids", "metabric"]:

    print("=======================")
    print("Evaluate ", ref_df)

    df, duration_col, event_col, time_horizons = get_dataset(ref_df)
    train_df, test_df = train_test_split(df, random_state=0)

    df_hash = dataframe_hash(train_df)

    X = train_df.drop(columns=[duration_col, event_col])
    T = train_df[duration_col]
    E = train_df[event_col]

    Xtest = train_df.drop(columns=[duration_col, event_col])
    Ttest = train_df[duration_col]
    Etest = train_df[event_col]

    cindex = []
    brier = []

    for seed in range(3):
        model_bkp = out_dir / f"{df_hash}_gmm_{seed}.bkp"

        if model_bkp.exists():
            syn_df = load_from_file(model_bkp)
        else:
            generator = TabularGMM(random_state=seed, components=100)

            generator.fit(X, T, E)

            Xsample, Tsample, Esample = generator.generate()

            syn_df = Xsample.copy()
            syn_df[duration_col] = Tsample
            syn_df[event_col] = Esample

            save_to_file(model_bkp, syn_df)

        Xsample = syn_df.drop(columns=[duration_col, event_col])
        Tsample = syn_df[duration_col]
        Esample = syn_df[event_col]

        model = RiskEstimation().get("cox_ph")
        model.fit(Xsample, Tsample, Esample)

        score = evaluate_survival_estimator(
            [model] * 3,
            Xtest,
            Ttest,
            Etest,
            time_horizons=time_horizons,
            pretrained=True,
        )
        cindex.append(score["clf"]["c_index"][0])
        brier.append(score["clf"]["brier_score"][0])

    cidx_str = print_score(generate_score(cindex))
    brier_str = print_score(generate_score(brier))
    print(ref_df, cidx_str, brier_str)
    scores.append((ref_df, cidx_str, brier_str))

In [ ]:
import tabulate

tabulate.tabulate(scores, headers=headers, tablefmt="html")

In [ ]:
from pathlib import Path

from sklearn.model_selection import train_test_split
from synthcity.plugins.core.models.survival_analysis.benchmarks import (
    evaluate_survival_model,
)
from synthcity.utils.evaluation import generate_score, print_score
from synthcity.utils.serialization import dataframe_hash, load_from_file, save_to_file

out_dir = Path("workspace")

log.remove()
headers = ["dataset", "cindex", "brier score"]

scores = []
for ref_df in ["aids", "metabric"]:

    print("=======================")
    print("Evaluate ", ref_df)

    df, duration_col, event_col, time_horizons = get_dataset(ref_df)
    train_df, test_df = train_test_split(df, random_state=0)

    df_hash = dataframe_hash(train_df)

    X = train_df.drop(columns=[duration_col, event_col])
    T = train_df[duration_col]
    E = train_df[event_col]

    Xtest = test_df.drop(columns=[duration_col, event_col])
    Ttest = test_df[duration_col]
    Etest = test_df[event_col]

    cindex = []
    brier = []

    for seed in range(3):
        model_bkp = out_dir / f"{df_hash}_gmm_{seed}.bkp"

        if model_bkp.exists():
            syn_df = load_from_file(model_bkp)
        else:
            generator = TabularGMM(random_state=seed, components=100)

            generator.fit(X, T, E)

            Xsample, Tsample, Esample = generator.generate()

            syn_df = Xsample.copy()
            syn_df[duration_col] = Tsample
            syn_df[event_col] = Esample

            save_to_file(model_bkp, syn_df)

        Xsample = syn_df.drop(columns=[duration_col, event_col])
        Tsample = syn_df[duration_col]
        Esample = syn_df[event_col]

        model = RiskEstimation().get("cox_ph")
        model.fit(Xsample, Tsample, Esample)

        score = evaluate_survival_model(
            [model] * 3,
            Xtest,
            Ttest,
            Etest,
            time_horizons=time_horizons,
            pretrained=True,
        )
        cindex.append(score["clf"]["c_index"][0])
        brier.append(score["clf"]["brier_score"][0])

    cidx_str = print_score(generate_score(cindex))
    brier_str = print_score(generate_score(brier))
    print(ref_df, cidx_str, brier_str)
    scores.append((ref_df, cidx_str, brier_str))

In [ ]:
import tabulate

tabulate.tabulate(scores, headers=headers, tablefmt="html")